# Loading in Data From Compressed File
I load data from the compressed file and use limited memory to do so. I select only the subset of columns I need.

In [1]:
# <INPUT Required> Put path to compressed file here
fpath = "/home/jake/Dropbox/classes/3rd Year/Teaching/ECON210_Fall2017/Data/psid-base.zip"

# Collecting information about dataset without reading whole thing
df.temp = read.table(unz(fpath, "psid-base.csv"), nrows=2, header=T, sep=",")
ncols = dim(df.temp)[2]

cols.all = colnames(df.temp)

# <INPUT Required> Insert desired variables here
yvars = c("inc_labor", "hours") # variables with year suffix
ivars = c("birthyear", "f_id", "id") # variables without year suffix (keep birthyear and f_id)

years = 1968:2011
cols = ivars
for (y in years){
    for (v in yvars) {
        if (paste(v, y, sep="") %in% cols.all) {
            cols = c(cols, paste(v, y, sep=""))
        }
    }
}

if (length(match(cols, cols.all)) != length(cols)) {
    print('Some columns not found, check columns')
}

cols.arg = rep("NULL", ncols)
for (c in 1:ncols) {
    if (c %in% match(cols, cols.all)){
        cols.arg[c] = "numeric"
    }
}

# Reading in whole dataset for selected columns
df.yr = read.table(unz(fpath, "psid-base.csv"), header=T, sep=",",
    colClasses=cols.arg)

You can use this dataset above. Below, I will construct a dataset that has variables by age instead of by year.

# Converting Variables from by Year to by Age

In [2]:
df.yr.cc = df.yr[complete.cases(df.yr$birthyear), ]

In [3]:
# <INPUT Required> Desired age range
ages = 25:50

df.age = df.yr.cc[,ivars]
for (v in yvars) {
    for (a in ages){
        avar = paste(v, "_age", a, sep="")
        df.age[,avar] = NaN
        for (y in years) {
            yvar = paste(v, y, sep="")
            if (yvar %in% cols.all) {
                df.age[df.age$birthyear==y - a, avar] = df.yr.cc[df.yr.cc$birthyear==y - a, yvar]
            }
        }
    }
}

# Reshaping Age and Year Datasets into Panel Datasets
## Panel Data set by Age

In [4]:
library(data.table)

df.age.panel = melt(as.data.table(df.age), 
     measure = patterns(sapply(yvars, function(y){paste(y, "_age", sep="")})), ## identify columns by patterns
     value.name = yvars)[    ## specify the resulting variable names
       ## melt creates a numeric "variable" value. Replace with factored labels
       , variable := factor(variable, labels=ages)][]
names(df.age.panel)[names(df.age.panel) == "variable"] = "age"
df.age.panel = as.data.frame(df.age.panel)

In [5]:
# first 10 observations with complete cases in yvars
df.age.panel[complete.cases(df.age.panel[,yvars]),][1:10,]

,birthyear,f_id,id,age,inc_labor,hours
1,1970,1335001,1335030,25,41340.0,2450
2,1955,1335001,1335006,25,46240.0,2382
3,1952,1335001,1335004,25,0.0,0
5,1962,1335001,1335010,25,42000.0,2686
6,1958,1335001,1335008,25,11950.0,800
7,1959,1335001,1335009,25,27571.6,2640
8,1953,1335001,1335005,25,365.0,34
9,1956,1335001,1335007,25,41920.0,1840
10,1964,1335001,1335011,25,8640.0,1750
11,1967,1411001,1411013,25,27582.5,1485


## Panel Data Set by Year

In [6]:
df.yr.full = df.yr

for (y in years) {
    for (v in yvars) {
        if (!(paste(v, y, sep="") %in% cols.all)) {
            df.yr.full[, paste(v, y, sep="")] = NaN
        }
    }
}
df.yr.panel = melt(as.data.table(df.yr.full), 
     measure = patterns(yvars), ## identify columns by patterns
     value.name = yvars)[    ## specify the resulting variable names
       ## melt creates a numeric "variable" value. Replace with factored labels
       , variable := factor(variable, labels=years)][]
names(df.yr.panel)[names(df.yr.panel) == "variable"] = "year"
df.yr.panel = as.data.frame(df.yr.panel)

In [7]:
# first 10 observations with complete cases in yvars
df.yr.panel[complete.cases(df.yr.panel[,yvars]),][1:10,]

,id,birthyear,f_id,year,inc_labor,hours
604,1554001,1895,NA,1968,0.0,0
607,2688001,1912,NA,1968,47880.0,2250
622,1101001,NA,NA,1968,23940.0,2256
633,5912002,1946,NA,1968,13167.0,1290
638,5861001,1948,NA,1968,7660.8,1200
650,2829001,1934,NA,1968,24624.0,4732
652,1677002,1941,NA,1968,12996.0,1920
665,717001,NA,NA,1968,54993.6,4080
677,2611002,1934,NA,1968,6840.0,480
680,1227001,1916,NA,1968,44460.0,2460
